In [1]:
import requests
from sphinx.ext.intersphinx import InventoryFile
from io import BytesIO
from urllib.parse import urljoin

def download_objects_inv(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return None

def parse_objects_inv(data, base_url):
    # Use InventoryFile to parse the inventory
    inv_file = InventoryFile.load(BytesIO(data), base_url, urljoin)
    
    inventory = {}
    for project_name, project_data in inv_file.items():
        inventory[project_name] = {}
        for name, (project, version, uri, display_name) in project_data.items():
            inventory[project_name][name] = (uri, display_name)
    return inventory

def save_inventory_to_file(inventory, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for category, items in inventory.items():
            file.write(f"\nCategory: {category}\n")
            for name, (location, dispname) in items.items():
                file.write(f"  {name} -> {location} ({dispname})\n")

def print_inventory(url, filename):
    data = download_objects_inv(url + 'objects.inv')
    if data:
        inventory = parse_objects_inv(data, url)
        save_inventory_to_file(inventory, filename)
        print(f"Inventory saved to {filename}")

# URL of the objects.inv file
url = 'https://numpy.org/devdocs/'

# Output file name
output_file = 'inventory.txt'

# Print the inventory
print_inventory(url, output_file)


Inventory saved to inventory.txt
